<img src="https://upload.wikimedia.org/wikipedia/commons/d/df/Logo_UNIR.png" width="350" height="175">

# *TFM: Comparación y optimización de algoritmos de Machine Learning sobre el éxito de campañas de marketing bancarias*

Autor: ***Jorge López Pérez***

***

## ***2. Limpieza de datos***

A lo largo de este cuaderno, realizaremos la limpieza de nuestro conjunto de datos.

In [13]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [16]:
# leemos los datos
data = pd.read_csv('https://raw.githubusercontent.com/JorgeLopez88/TFM/main/data/raw_data.csv',sep=';')
data = data.replace('unknown', np.nan) #sustituimos los unknown por nan
data.shape

(41188, 21)

***

#### 2.1 Valores únicos por columna

In [3]:
data.nunique()

age                 78
job                 11
marital              3
education            7
default              2
housing              2
loan                 2
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

(Vemos que no hay atributos con 1 solo valor único)

(Default lo dejaremos y evaluaremos más adelante a pesar de que una de sus categorías representa el 99,99% de las isntancias)

***

#### 2.2 Comprobación tipos de datos asociados

In [4]:
data.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

(Los tipos asociados son correctos)

***

#### 2.3 Uniformidad valores nulos

Categorías de los atributos categóricos:

In [5]:
categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for variable in categoricas:
  print(data[variable].value_counts())

  print('')

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
Name: count, dtype: int64

marital
married     24928
single      11568
divorced     4612
Name: count, dtype: int64

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
illiterate                18
Name: count, dtype: int64

default
no     32588
yes        3
Name: count, dtype: int64

housing
yes    21576
no     18622
Name: count, dtype: int64

loan
no     33950
yes     6248
Name: count, dtype: int64

contact
cellular     26144
telephone    15044
Name: count, dtype: int64

month
may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      18

(Los atributos numéricos no tienen valores nulos. Respecto a los categóricos, vemos que las categorías anteriores no tienen más categorías susceptibles de ser nulas, a parte de la ya conocida categoría 'unknown')

***

#### 2.4 Instancias duplicadas

Vemos las instancias duplicadas:

In [6]:
data[data.duplicated()]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1266,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
12261,36,retired,married,NaN,no,no,no,telephone,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
14234,27,technician,single,professional.course,no,no,no,cellular,jul,mon,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
16956,47,technician,divorced,high.school,no,yes,no,cellular,jul,thu,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
18465,32,technician,single,professional.course,no,yes,no,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1,no
20216,55,services,married,high.school,NaN,no,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,no
20534,41,technician,married,professional.course,no,yes,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no
25217,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,no
28477,24,services,single,high.school,no,yes,no,cellular,apr,tue,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.423,5099.1,no
32516,35,admin.,married,university.degree,no,yes,no,cellular,may,fri,...,4,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no


Las eliminamos y dejamos de esta forma una sola instancia única para cada duplicado:

In [8]:
data_sin_duplicados = data.drop_duplicates()
data_sin_duplicados.shape

(41176, 21)

(Vemos que nos quedamos con 41.176 instancias, tras eliminar las 12 duplicadas)

***

#### 2.5 Valores erróneos

(Las categorías podemos ver en el punto 2.3 que son coherentes dentro de su contexto)

(Respecto a los atributos numéricos tenemos los siguientes rangos):

In [10]:
numericas = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

for atributo in numericas:
  print('Min: ', data_sin_duplicados[atributo].min())
  print('Max: ', data_sin_duplicados[atributo].max())
  print('--------------')
  print('')

Min:  17
Max:  98
--------------

Min:  0
Max:  4918
--------------

Min:  1
Max:  56
--------------

Min:  0
Max:  999
--------------

Min:  0
Max:  7
--------------

Min:  -3.4
Max:  1.4
--------------

Min:  92.201
Max:  94.767
--------------

Min:  -50.8
Max:  -26.9
--------------

Min:  0.634
Max:  5.045
--------------

Min:  4963.6
Max:  5228.1
--------------



Los valores son correctos. El único que nos puede extrañar es el valor 999 en pdays pero recordemos que es como se identifica cuando no se ha contactado ese cliente previamente nunca.

***

Guardamos los datos limpios:

In [11]:
data_sin_duplicados.to_csv('clean_data.csv', index=False)